In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:04 of 0:12:04 remaining]
Chain 1:  10% [0:00:54 of 0:00:60 remaining]
Chain 1:  20% [0:00:56 of 0:01:10 remaining]
Chain 1:  30% [0:00:52 of 0:01:15 remaining]
Chain 1:  40% [0:00:46 of 0:01:17 remaining]
Chain 1:  50% [0:00:39 of 0:01:17 remaining]
Chain 1:  60% [0:00:31 of 0:01:17 remaining]
Chain 1:  70% [0:00:23 of 0:01:18 remaining]
Chain 1:  80% [0:00:15 of 0:01:17 remaining]
Chain 1:  90% [0:00:08 of 0:01:17 remaining]
Chain 1: 100% [0:00:00 of 0:01:17 remaining]

Chain 2:   0% [0:00:17 of 0:00:17 remaining]
Chain 2:  10% [0:00:48 of 0:00:53 remaining]
Chain 2:  20% [0:00:53 of 0:01:06 remaining]
Chain 2:  30% [0:00:48 of 0:01:08 remaining]
Chain 2:  40% [0:00:42 of 0:01:09 remaining]
Chain 2:  50% [0:00:35 of 0:01:10 remaining]
Chain 2:  60% [0:00:28 of 0:01:10 remaining]
Chain 2:  70% [0:00:21 of 0:01:11 remaining]
Chain 2:  80% [0:00:14 of 0:01:11 remaining]
Chain 2:  90% [0:00:07 of 0:01:11 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.5089 114.08 0.906903; 25.0269 114.08 0.906903; … ; 29.9137 111.503 0.95969; 30.9716 113.51 0.918744]

[26.0006 112.149 0.944307; 28.2906 112.19 0.945082; … ; 25.6726 113.855 0.908003; 27.0773 113.855 0.908003]

[27.0828 112.043 0.951091; 27.3751 117.458 0.834313; … ; 25.4881 112.584 0.923903; 25.4056 112.594 0.94008]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD        Naive SE       MCSE        ESS  
     s2  26.1696678 2.01538767 0.01226525876 0.0198827841 9000.000
beta[1] 113.8790522 1.95229946 0.01188131614 0.0385508757 2564.628
beta[2]   0.9050053 0.04285097 0.00026078271 0.0008399617 2602.570

Quantiles:
            2.5%       25.0%       50.0%       75.0%        97.5%   
     s2  22.4777551  24.7794017  26.0744570  27.4608463  30.39217044
beta[1] 110.0974328 112.5725128 113.8730062 115.1498225 117.73627284
beta[2]   0.8203202   0.8764732   0.9053482   0.9337954   0.98867997



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*